In [12]:
import logging
import os
import tensorflow as tf
import gym
from baselines import logger
from baselines.common import set_global_seeds
from baselines import bench
from baselines.acktr.acktr_cont import learn
from baselines.acktr.policies import GaussianMlpPolicy,CnnPolicy
from baselines.acktr.value_functions import NeuralNetValueFunction

In [4]:
import gym
import gym_minecraft
from MinecraftGym import MinecraftWrapper

In [7]:
from keras import backend as K
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Activation,GRU,Input,LSTM,Conv2D,Flatten
from keras.optimizers import RMSprop,Adam,TFOptimizer
from keras.callbacks import TensorBoard

In [6]:
from baselines.acktr import kfac

In [8]:
pre_env = gym.make("MinecraftCliffWalking1-v0")
pre_env.init(videoResolution=[400,400],allowContinuousMovement=["move", "turn", "strafe"])
env = MinecraftWrapper(pre_env,scale=1/5)

In [6]:
## create an image based value function using CNN

model = Sequential()
model.add(Conv2D(32,(8,8),input_shape=env.observation_space.shape,activation='relu'))
model.add(Conv2D(64,(4,4),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Flatten())
model.add(Dense(60,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(output_dim=env.action_space.n,activation='softmax'))

/home/ubuntu/miniconda2/envs/casper/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=6, activation="softmax")`


In [15]:
class CNNValueFunction(object):
    def __init__(self, ob_dim, ac_dim): #pylint: disable=W0613
        model = Sequential()
        model.add(Conv2D(32,(8,8),input_shape=ob_dim,activation='relu'))
        model.add(Conv2D(64,(4,4),activation='relu'))
        model.add(Conv2D(64,(3,3),activation='relu'))
        model.add(Flatten())
        model.add(Dense(60,activation='relu'))
        model.add(Dense(32,activation='relu'))
        model.add(Dense(output_dim=env.action_space.n,activation='softmax'))
        optim = kfac.KfacOptimizer(learning_rate=0.001, cold_lr=0.001*(1-0.9), momentum=0.9, \
                                    clip_kl=0.3, epsilon=0.1, stats_decay=0.95, \
                                    async=1, kfac_update=2, cold_iter=50, max_grad_norm=1.0)
        self.model = model.compile(loss="mean_squared_error",optimizer=TFOptimizer(optim))
    def predict(self, path):
        return self.model.predict(path)
    def fit(self, paths, targvals):
        '''
        X = np.concatenate([self._preproc(p) for p in paths])
        y = np.concatenate(targvals)
        logger.record_tabular("EVBefore", common.explained_variance(self._predict(X), y))
        for _ in range(25): self.do_update(X, y)
        logger.record_tabular("EVAfter", common.explained_variance(self._predict(X), y))
        '''
        self.model.fit(paths,targvals)


In [26]:

def train(num_timesteps):
    #env=gym.make(env_id)
    #env = bench.Monitor(env, logger.get_dir() and os.path.join(logger.get_dir(), str(rank)))
    #set_global_seeds(seed)
    #env.seed(seed)
    gym.logger.setLevel(logging.WARN)

    with tf.Session(config=tf.ConfigProto()) as sess:
        ob_dim = env.observation_space.shape
        ac_dim = env.action_space.n
        with tf.variable_scope("vf"):
            vf = CNNValueFunction(ob_dim,ac_dim)
        with tf.variable_scope("pi"):
            policy = CnnPolicy(sess,env.observation_space, env.action_space,1,64,1)

        learn(env, policy=policy, vf=vf,
            gamma=0.99, lam=0.97, timesteps_per_batch=2500,
            desired_kl=0.002,
            num_timesteps=num_timesteps, animate=False)

    env.close()

In [27]:
train(int(1e6))

/home/ubuntu/miniconda2/envs/casper/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=6, activation="softmax")`


AttributeError: 'CnnPolicy' object has no attribute 'update_info'